# Module 1: Introduction to Exploratory Analysis 

What we'll be doing in this notebook:
-----

 1.  Checking variable type
 2.  Checking for missing variables 
 3.  Number of observations in the dataset
 4.  Descriptive statistics

### Import packages

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil.parser

# The command below means that the output of multiple commands in a cell will be output at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The command below tells jupyter to display up to 80 columns, this keeps everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

# Show figures in notebook
%matplotlib inline

### Import dataset

We read in our dataset

In [29]:
path = '../data/'
filename = 'loans.csv'
df = pd.read_csv(path+filename)

In the cell below, we take a random sample of 2 rows to get a feel for the data.

In [30]:
df.sample(2)

,id_number,loan_amount,status,funded_date,funded_amount,location_country_code,activity,description_texts_en,use
237,874800,325,funded,2015-04-30T04:41:34Z,325,SS,Farming,NaN,NaN
49,908034,5050,funded,2015-07-09T06:55:03Z,5050,CG,Fish Selling,Francoise is a member of the community bank gr...,to purchase baskets of fish for resale.


### 1) Type Checking
<a id='type_check'></a>

Type is very important in Python programing, because it affects the types of functions you can apply to a series. There are a few different types of data you will see regularly (see [this](https://en.wikibooks.org/wiki/Python_Programming/Data_Types) link for more detail):
* **int** - a number with no decimal places. example: loan_amount field
* **float** - a number with decimal places. example: partner_id field
* **str** - str is short for string. This type formally defined as a sequence of unicode characters. More simply, string means that the data is treated as a word, not a number. example: sector
* **boolean** - can only be True or False. There is not currently an example in the data, but we will be creating a gender field shortly.
* **datetime** - values meant to hold time data. Example: posted_date

Let's check the type of our variables using the examples we saw in the cell above.

In [31]:
# Here are all of the columns
df.columns.tolist()

['id_number',
 'loan_amount',
 'status',
 'funded_date',
 'funded_amount',
 'location_country_code',
 'activity',
 'description_texts_en',
 'use']

In [32]:
# Find the dtype, aka datatype, for a column
df['id_number'].dtype

dtype('int64')

In [19]:
# Try this - Pick a couple of columns and check their type on your own


### 2) Do I have missing values?

<a id='missing_check'></a>

If we have missing data, is the missing data at random or not? If data is missing at random, the data distribution is still representative of the population. You can probably ignore the missing values as an inconvenience. However, if the data is systematically missing, the analysis you do may be biased. You should carefully consider the best way to clean the data, it may involve dropping some data.

We want to see how many values are missing in certain variable columns. One way to do this is to count the number of null observations. 

For this, we wrote a short function to apply to the dataframe. 

We print out the first few observations, but you can remove the .head() to print out all columns. 

In [33]:
#Create a new function:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
## Check how many are missing by column, and then check which ones have any missing values
print(df.apply(num_missing, axis=0).where(lambda x : x != 0).dropna().head(20)) 
#axis=0 defines that function is to be applied on each column

Missing values per column:
funded_date              126.0
location_country_code     10.0
description_texts_en      16.0
use                       16.0
dtype: float64


### 3) Sanity Checks
<a id='obs_check'></a>

**Does the dataset match what you expected to find?**
- is the range of values what you would expect. For example, are all loan_amounts above 0.
- do you have the number of rows you would expect
- is your data for the date range you would expect. For example, is there a strange year in the data like 1880.
- are there unexpected spikes when you plot the data over time


In the command below we find out the number of loans and number of columns by using the function shape. You can also use len(df.index) to find the number of rows.

In [37]:
print(f'There are {df.shape[0]} observations and {df.shape[1]} features')

There are 282 observations and 9 features


Remember, each row is an observation and each column is a potential feature. 

Remember we need large about of data for machine learning.

### 4) Descriptive statistics of the dataset

<a id='desc_stats'></a>

The "describe" command below provides key summary statistics for each numeric column.

In [38]:
df.describe()

,id_number,loan_amount,funded_amount
count,2.820000e+02,282.000000,282.000000
mean,1.411887e+06,3226.152482,2421.808511
std,2.836500e+05,8194.704207,8222.169629
min,5.143710e+05,50.000000,0.000000
25%,1.308959e+06,325.000000,62.500000
50%,1.566896e+06,862.500000,300.000000
75%,1.570511e+06,3568.750000,1581.250000
max,1.571572e+06,80000.000000,80000.000000


In order to get the same summary statistics for categorical columns (string) we need to do a little data wrangling. The first line of code filters for all columns that are a data type object. As we know from before this means they are considered to be a string. The final row of code provides summary statistics for these character fields.

In [39]:
categorical = df.dtypes[df.dtypes == "object"].index
df[categorical].describe()

,status,funded_date,location_country_code,activity,description_texts_en,use
count,282,156,272,282,266,266
unique,2,152,28,63,256,239
top,funded,2018-07-21T17:50:44Z,CD,Retail,Zawadi has been a customer of the microfinance...,to pay for a stove.
freq,156,3,20,31,2,6


In the table above, there are 4 really useful fields: 

1) **count** - total number of fields populated (Not empty). 

2) **unique** - tells us how many different unique ways this field is populated. For example 4 in description.languages tells us there are 4 different language descriptions. 

3) **top** - tells us the most popular data point. For example, the top activity in this dataset is Farming which tells us most loans are in Farming.

4) **freq** - tells us that how frequent the most popular category is in our dataset. For example, 'en' (English) is the language almost all descriptions (description.languages) are written in (118,306 out of 118,316).

### Moving on

In the next section, we move on to exploratory data analysis (EDA). We'll plot this data!

<br>
<br> 
<br>

----